In [1]:
import pandas as pd
import nltk
import numpy as np
import re

from nltk.corpus import stopwords


df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv",encoding='latin-1')
df.head(5900)


dataset_size = len(df)  
num_features = len(df.columns)  

print("Dataset size:", dataset_size)
print("Number of features:", num_features)

df = df[['v2','v1']]
df.rename(columns={'v2':'messages','v1':'Label'},inplace=True)
df.head(5572)

Dataset size: 5572
Number of features: 5


,messages,Label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam
5568,Will Ì_ b going to esplanade fr home?,ham
5569,"Pity, * was in mood for that. So...any other s...",ham
5570,The guy did some bitching but I acted like i'd...,ham


preprocessing the dataset


In [2]:
df.isnull().sum()

messages    0
Label       0
dtype: int64

In [3]:
Stopwords = set(stopwords.words('english'))
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^0-9a-zA-Z]',' ',text)
    #remove extra spaces
    text = re.sub(r'\s+',' ',text)
    #remove stopwords
    text = "".join (word for word in text.split() if word not in Stopwords)
    return text
    

In [4]:
df['clean_text']= df['messages'].apply(clean_text)
df.head()

,messages,Label,clean_text
0,"Go until jurong point, crazy.. Available only ...",ham,gojurongpointcrazyavailablebugisngreatworldlae...
1,Ok lar... Joking wif u oni...,ham,oklarjokingwifuoni
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,freeentry2wklycompwinfacupfinaltkts21stmay2005...
3,U dun say so early hor... U c already then say...,ham,udunsayearlyhorucalreadysay
4,"Nah I don't think he goes to usf, he lives aro...",ham,nahthinkgoesusflivesaroundthough


input split

In [5]:
X = df['clean_text']
y = df['Label']

model training

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

def classify(model, X, y):
    # train test split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True, stratify=y)
    # model training
    pipeline_model = Pipeline([('vect', CountVectorizer()),
                               ('tfidf',TfidfTransformer()),
                               ('clf', model)])
    pipeline_model.fit(x_train, y_train)
    
    print('Accuracy:', pipeline_model.score(x_test, y_test)*100)
    

    #y_pred = pipeline_model.predict(x_test)
    # print(classification_report(y_test, y_pred))

In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classify(model, X, y)

Accuracy: 86.57573582196699


In [8]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
classify(model, X, y)

Accuracy: 86.71931083991386


In [9]:
from sklearn.svm import SVC
model = SVC(C=3)
classify(model, X, y)

Accuracy: 89.44723618090453


In [10]:

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
classify(model, X, y)

Accuracy: 89.44723618090453
